In [1]:
import numpy as np

In [2]:
from dgh import upper

In [ ]:
#from smart_dgh import calculate_dgh

In [3]:
def get_points_circle(n, R, x_center, y_center, z_center):
    x = np.random.uniform(x_center - R, x_center + R, n)
    y = np.random.uniform(y_center - np.sqrt(np.square(R)-np.square(x-x_center)), y_center + np.sqrt(np.square(R)-np.square(x-x_center)), n)
    z_pos_sqrt = np.sqrt(np.square(R)-np.square(x-x_center)-np.square(y-y_center))
    random_adj = np.random.choice([-1,1],len(x))
    z = random_adj * z_pos_sqrt + z_center
    return x,y,z

In [62]:
points_X = get_points_circle(35, 30, 0,0,0)

Y_x = points_X[0] + 61
Y_y = points_X[1] + 61
Y_z = points_X[2] + 61
points_Y = (Y_x, Y_y, Y_z)


coords_X = np.vstack(points_X).T
coords_Y = np.vstack(points_Y).T

X = np.linalg.norm(coords_X[:, None] - coords_X, axis=2)
Y = np.linalg.norm(coords_Y[:, None] - coords_Y, axis=2)

In [5]:
import numpy as np
def generate_symmetric_matrix(size, min_val=1, max_val=100):

    matrix = np.random.randint(min_val, max_val + 1, size=(size, size))

    symmetric_matrix = (matrix + matrix.T) / 2

    np.fill_diagonal(symmetric_matrix, 0.0)
    return symmetric_matrix


In [35]:
X = generate_symmetric_matrix(5, max_val= 200)
Y = X.copy()
X

array([[  0. , 166. ,  75. , 101. ,  78. ],
       [166. ,   0. , 173. , 144.5, 103.5],
       [ 75. , 173. ,   0. , 154. , 145.5],
       [101. , 144.5, 154. ,   0. ,  55.5],
       [ 78. , 103.5, 145.5,  55.5,   0. ]])

In [ ]:
import numpy as np
import random

from mappings import rnd_S, center, S_to_fg, S_to_R, is_in_bimapping_polytope
from fw import make_frank_wolfe_solver
from spaces import diam, rad, arrange_distances
from constants import DEFAULT_SEED, C_SEARCH_GRID
from dgh import dis, upper



def fg_to_R_float(f, g):
    """
    Represents a mapping pair as a binary row-stochastic block matrix.

    :param f: mapping in X🠖Y (1d-array)
    :param g: mapping in Y🠖X (1d-array)
    :return: mapping pair representation R ∈ ℛ (2d-array)
    """
    n, m = len(f), len(g)
    nxn_zeros, mxm_zeros = (np.zeros((size, size), dtype=float) for size in [n, m])

    F = np.identity(m, dtype=float)[f]
    G = np.identity(n, dtype=float)[g]

    R = np.block([[F, nxn_zeros], [mxm_zeros, G]])

    return R

def update_fg(best_f, best_g, set_f, set_g, change_number = 1):
   
    f = best_f.copy()
    g = best_g.copy()


    pairs_f = random.sample(set_f, min(len(set_f),change_number))
    pairs_g = random.sample(set_g, min(len(set_g),change_number))

    for pair_f in pairs_f:
       if (f[pair_f[0]] != f[pair_f[1]]):
          continue
       new_destination = np.random.choice(len(g), 2, replace=False)
       f[pair_f[0]],f[pair_f[1]] = new_destination[0], new_destination[1]
    
    for pair_g in pairs_g:
       if (g[pair_g[0]] != g[pair_g[1]]):
          continue
       new_destination = np.random.choice(len(f), 2, replace=False)
       g[pair_g[0]],g[pair_g[1]] = new_destination[0], new_destination[1]



    return f,g
    
def calculate_dgh(X, Y, budget_for_start = 1000, search_budget = 2000, change_number = 1, tol = 1e-16, return_fg = False):

    n, m = len(X), len(Y)
    
    diam_X, diam_Y = map(diam, [X, Y])
    d_max = max(diam_X, diam_Y)

    rad_X, rad_Y = map(rad, [X, Y])
    lb = max(0, abs(diam_X - diam_Y)/2, abs(rad_X - rad_Y)/2)

    X, Y = map(lambda Z: Z.copy() / d_max, [X, Y])
    lb /= d_max

    start_dgh, best_f, best_g, c = upper(X,Y,iter_budget=budget_for_start, return_fg = True)

    #print(fg_to_R_float(best_f, best_g))
    #print(best_f)

    start_dgh *= d_max
    best_dis_R = start_dgh * 2

    if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
        res = (best_dis_R/2, best_f, best_g) if return_fg else best_dis_R/2
        return res
    
    set_f = []
    set_g = []

    for i in range(len(best_f)):
       for j in range(i+1, len(best_f)):
          if best_f[i] == best_f[j]:
             set_f.append((i, j))
    
    for i in range(len(best_g)):
       for j in range(i+1, len(best_g)):
          if best_g[i] == best_g[j]:
             set_g.append((i, j))

    print(len(set_f), len(set_g))

    fw = make_frank_wolfe_solver(X, Y, c, tol=tol)

    while (search_budget > 0):
       
        f, g = update_fg(best_f, best_g, set_f, set_g, change_number)

        S0 = fg_to_R_float(f, g)

        #print(f)
        
        S, used_iter = fw(S0=S0, max_iter=search_budget)

        search_budget -= used_iter

        R = S_to_R(S, n, m)

        
        dis_R = dis(R, X, Y) * d_max
        
        #print(dis_R, best_dis_R)
        
        if dis_R != best_dis_R:
           print(dis_R, best_dis_R)

        if dis_R < best_dis_R:
            best_f, best_g = map(list, S_to_fg(S, n, m))
            best_dis_R = dis_R
        
        if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
           break
    
    res = (best_dis_R/2, best_f, best_g) if return_fg else best_dis_R/2

    return res

In [118]:
import numpy as np
from functools import partial

from mappings import is_row_stoch, fg_to_R
from spaces import arrange_distances


def solve_frank_wolfe(obj, grad, find_descent_direction, minimize_obj_wrt_gamma, S0,
                      tol=1e-16, max_iter=np.inf, verbose=0, d = None, use_taboo = True):

    S = S0.copy()
    for iter in range(max_iter):
        # Find the Frank-Wolfe direction.
        grad_at_S = grad(S)
        R = find_descent_direction(grad_at_S)
        D = R - S

        # Find γ ∈ [0, 1] defining how much to go in the decided direction.
        global_gamma = minimize_obj_wrt_gamma(S, D)
        critical_gammas = {0, 1}
        if 0 < global_gamma < 1:
            critical_gammas.add(global_gamma)
        gamma = min(critical_gammas, key=lambda x: obj(S + x*D))

        if verbose > 2:
            print(f'  iter {iter}: σ(S)={obj(S):.4f}, γ={gamma:.5f}')

        # Move S towards R by γ, i.e. to (1-γ)S + γR.
        S += gamma * D

        if (use_taboo and iter > 1 and iter % 2 == 0):
            hash = tuple(S.argmax(axis = 1))
            if hash in d:
                print('nothing to find here')
                break

        # Stop if the rate of descent is too small or if the line search stalls.
        if np.sum(-grad_at_S * D) < tol or np.isclose(gamma, 0):
            break

    if use_taboo:
        hash = tuple(S.argmax(axis = 1))
        d[hash] = 1
        
    return S, iter + 1


def make_frank_wolfe_solver(X, Y, c, **kwargs):
    
    n, m = len(X), len(Y)

    # Define auxiliary function that is a component in the objective and its gradient.
    def dot_multiplicand(S):
        X__Y, Y__X, _Y_X = arrange_distances(X, Y)
        c_Y_X, c__Y_X = c**_Y_X,  c**-_Y_X

        return (c__Y_X @ S @ c_Y_X + c_Y_X @ S @ c__Y_X).T + \
            c**-X__Y @ S @ c**Y__X + c**X__Y @ S @ c**-Y__X

    # Smooth distortion σ as the objective.
    def obj(S):
        return np.sum(S * dot_multiplicand(S))

    # ∇σ.
    def grad(S):
        return 2 * dot_multiplicand(S)

    # To minimize〈R, ∇σ(S)〉over 𝓢 given S ∈ 𝓢, R must be a vertex of 𝓢.
    def find_descent_direction(grad_at_S):
        f = np.argmin(grad_at_S[:n, :m], axis=1)
        g = np.argmin(grad_at_S[n:, m:], axis=1)

        return fg_to_R(f, g)

    # To minimize σ(γ) = σ(S + γD), for line search.
    def minimize_obj_wrt_gamma(S, D):
        # Leverage that the objective is quadratic in γ, σ(γ) = aγ² + bγ + c.
        a = np.sum(D * dot_multiplicand(D))
        b = np.sum(D * dot_multiplicand(S)) + np.sum(S * dot_multiplicand(D))
        with np.errstate(divide='ignore', invalid='ignore'):
            global_gamma = np.divide(-b, 2*a)

        return global_gamma

    fw = partial(solve_frank_wolfe, obj, grad, find_descent_direction,
                 minimize_obj_wrt_gamma, **kwargs)

    return fw


In [127]:
def calculate_dgh_v2(X, Y, budget_for_start = 1000, search_budget = 2000, change_number = 1, tol = 1e-16, return_fg = False, restart_nums = 10):

    '''
    with taboo search +
    with restarts +
    '''

    n, m = len(X), len(Y)
    
    diam_X, diam_Y = map(diam, [X, Y])
    d_max = max(diam_X, diam_Y)

    rad_X, rad_Y = map(rad, [X, Y])
    lb = max(0, abs(diam_X - diam_Y)/2, abs(rad_X - rad_Y)/2)

    X, Y = map(lambda Z: Z.copy() / d_max, [X, Y])
    lb /= d_max

    start_dgh, best_f, best_g, c = upper(X,Y,iter_budget=budget_for_start, return_fg = True)

    #print(fg_to_R_float(best_f, best_g))
    #print(best_f)

    start_dgh *= d_max
    best_dis_R = start_dgh * 2

    if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
        res = (best_dis_R/2, best_f, best_g) if return_fg else best_dis_R/2
        return res
    
    set_f = []
    set_g = []

    for i in range(len(best_f)):
       for j in range(i+1, len(best_f)):
          if best_f[i] == best_f[j]:
             set_f.append((i, j))
    
    for i in range(len(best_g)):
       for j in range(i+1, len(best_g)):
          if best_g[i] == best_g[j]:
             set_g.append((i, j))

    print(len(set_f), len(set_g))

    taboo_dict = dict()

    fw = make_frank_wolfe_solver(X, Y, c, tol=tol, d = taboo_dict)

    #restart_nums = 10 # search_budget // 100 #пока что так

    cur_f, cur_g = best_f.copy(), best_g.copy()

    while (restart_nums > 0):
       
       frank_budget = search_budget
       
       while (frank_budget > 0):
          
          f, g = update_fg(cur_f, cur_g, set_f, set_g, change_number)
          
          S0 = fg_to_R_float(f, g)
          
          S, used_iter = fw(S0=S0, max_iter=search_budget, use_taboo = True)
          
          frank_budget -= used_iter
          
          R = S_to_R(S, n, m)
          
          dis_R = dis(R, X, Y) * d_max
          
          if dis_R != best_dis_R:
             print(dis_R, best_dis_R)

          if dis_R < best_dis_R:
             
             best_f, best_g = map(list, S_to_fg(S, n, m))
             cur_f, cur_g = best_f.copy(), best_g.copy()
             best_dis_R = dis_R
        
          if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
             break
          
       if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
          break
       
       S0 = rnd_S(n, m)

       #print("RESTART start")
       S, _ = fw(S0=S0, max_iter=budget_for_start, use_taboo = False) #получаем новое вроде бы неплохое отображение
       #print("RESTART end")

       R = S_to_R(S, n, m)
       dis_R = dis(R, X, Y) * d_max

       cur_f, cur_g = map(list, S_to_fg(S, n, m))

       if dis_R < best_dis_R:
         best_f, best_g =  cur_f.copy(), cur_g.copy()
         best_dis_R = dis_R
         
       if np.isclose(best_dis_R, lb, atol=0.1): #atol = 1?
         break

       restart_nums -= 1

       print(f"{'restart', 10 - restart_nums}")
       
    
    res = (best_dis_R/2, best_f, best_g) if return_fg else best_dis_R/2

    return res

In [64]:
result = upper(X, Y, iter_budget=500, return_fg=True)
d_gh, f, g, c = result
d_gh

np.float64(9.982129780737243e-15)

In [65]:
result = upper(X, Y, iter_budget=100, return_fg=True)
d_gh, f, g, c = result
d_gh

np.float64(12.867148049952613)

In [130]:
result = calculate_dgh_v2(X, Y, budget_for_start = 100, search_budget = 50, change_number=20, return_fg=True, restart_nums=50)
d_gh, f, g = result
d_gh

20 19
15.91028027860233 25.734296099905226
24.36558138121449 15.91028027860233
25.734296099905226 15.91028027860233
25.734296099905222 15.91028027860233
('restart', -39)
23.090691836869542 15.91028027860233
21.314338889899656 15.91028027860233
25.734296099905226 15.91028027860233
nothing to find here
21.314338889899656 15.91028027860233
25.734296099905226 15.91028027860233
21.314338889899656 15.91028027860233
('restart', -38)
25.734296099905226 15.91028027860233
27.73416504041199 15.91028027860233
16.879680442648592 15.91028027860233
25.734296099905226 15.91028027860233
19.206508729108563 15.91028027860233
19.206508729108563 15.91028027860233
25.734296099905222 15.91028027860233
('restart', -37)
21.392287342867547 15.91028027860233
19.206508729108563 15.91028027860233
23.090691836869542 15.91028027860233
24.365581381214493 15.91028027860233
25.734296099905226 15.91028027860233
25.734296099905226 15.91028027860233
21.392287342867547 15.91028027860233
('restart', -36)
16.879680442648592 

np.float64(7.955140139301165)

In [175]:
import plotly.graph_objects as go
import numpy as np

def plot_incorrect_mappings(points_X, points_Y, f, g, title=""):
    x_X, y_X, z_X = points_X
    x_Y, y_Y, z_Y = points_Y
    n = len(x_X)

    incorrect_X = [i for i in range(n) if f[i] != i]
    incorrect_Y = [i for i in range(n) if g[i] != i]

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=x_X, y=y_X, z=z_X, mode='markers+text',
        text=[f'X{i}' for i in range(n)],
        marker=dict(color='blue', size=5),
        textposition='top center', name='X'
    ))
    fig.add_trace(go.Scatter3d(
        x=x_Y, y=y_Y, z=z_Y, mode='markers+text',
        text=[f'Y{i}' for i in range(n)],
        marker=dict(color='red', size=5),
        textposition='top center', name='Y'
    ))

    for i in incorrect_X:
        j = f[i]
        fig.add_trace(go.Scatter3d(
            x=[x_X[i], x_Y[j]], y=[y_X[i], y_Y[j]], z=[z_X[i], z_Y[j]],
            mode='lines', line=dict(color='blue', width=2, dash='dot'),
            name=f'f(X{i}) → Y{j}'
        ))

    for i in incorrect_Y:
        k = g[i]
        fig.add_trace(go.Scatter3d(
            x=[x_Y[i], x_X[k]], y=[y_Y[i], y_X[k]], z=[z_Y[i], z_X[k]],
            mode='lines', line=dict(color='red', width=2, dash='dot'),
            name=f'g(Y{i}) → X{k}'
        ))

    fig.update_layout(
        title=title,
        scene=dict(aspectmode='cube'),
        showlegend=True
    )

    return fig

In [176]:
plot_incorrect_mappings(points_X, points_Y, f, g, title="")